In [ ]:
# piecewise linear mapping not suitable for dnn (very deep neural networks tend to suffer from gradient vanishing)

# NN summary
### Components
+ Dense > BatchNormalization > DropOut >SoftMax

### Optimizer
+ Adam

### Loss
+ BinaryCrossEntropy

### Metric
+ Accuracy (%)

In [ ]:
## Note on accuracy:
# FCNN has too many parameters (to train)，10000 samples are not enough (overfitting or sensitive)，
# thus validation loss>> train los.
# GNN has much fewer parameters，10000 samples are onough，validation loss converges fast
# Although training loss/accuracy is similar for 3 models, validation loss looks different
# Also the validation results look different for DNN in different trails
# Somehow like the polynomial fitting problem?

In [ ]:
import pickle
from os import path
from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
from time import time
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))

Mounted at /content/drive
Tesla P100-PCIE-16GB


In [ ]:
!nvidia-smi

Sun Feb 28 23:14:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
filename='/content/drive/MyDrive/gnn/data/118ac_ISF.txt'
S_isf=pd.read_table(filename,sep=',',header=None).to_numpy() # ISF matrix

filename='/content/drive/MyDrive/gnn/data/118ac_fmax.txt'
f_max=pd.read_table(filename,sep=',',header=None).to_numpy() # flow limit

n_line = np.size(S_isf,0)

S = torch.from_numpy(S_isf).to(device) # ISF
f_max = torch.from_numpy(f_max).to(device) # flow limit

In [ ]:
# load data
x=np.load('/content/drive/MyDrive/gnn/data/ac118_p10_x_normal1.npy').transpose([2,1,0]).astype('float32')
y=np.load('/content/drive/MyDrive/gnn/data/ac118_p10_y_normal1.npy').transpose([1,0]).astype('float32')
W=np.load('/content/drive/MyDrive/gnn/data/ac118_p10_w.npy').astype('float32')

x=np.reshape(x,(x.shape[0],-1))
print(x.shape)
print(y.shape)
# train val test split
x_total,x_test,y_total,y_test=train_test_split(x,y,test_size=0.15,random_state=23)
kf=KFold(n_splits=10,shuffle=True)
for train_index,val_index in kf.split(x_total):
  x_train=x_total[train_index]
  y_train=y_total[train_index]
  x_val=x_total[val_index]
  y_val=y_total[val_index]
  break
# data loader
class Dataset(torch.utils.data.Dataset):
    def __init__(self,x,y,device):
        self.x=torch.from_numpy(x).float().to(device)
        self.y=torch.from_numpy(y).float().to(device)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx=idx.tolist()
        # Select sample
        return self.x[idx],self.y[idx]
params={'batch_size': 200,
        'shuffle': True,
        'num_workers': 0}
train=Dataset(x_train,y_train,device)
train_set=torch.utils.data.DataLoader(train,**params)
val=Dataset(x_val,y_val,device)
val_set=torch.utils.data.DataLoader(val,**params)

(10000, 354)
(10000, 118)


In [ ]:
class dnn(torch.nn.Module):
  def __init__(self,shape,dropout=0):
    super(dnn,self).__init__()
    layers=[]
    for idx in range(len(shape)-2):
      layers.extend([
        nn.Linear(shape[idx],shape[idx+1]),
        nn.ReLU(),
        nn.BatchNorm1d(shape[idx+1]),
        nn.Dropout(dropout)
      ])
    layers.append(nn.Linear(shape[-2],shape[-1]))
    self.features=nn.Sequential(*layers)
    # initialize
    for temp in self.features:
      if type(temp)==nn.Linear:
        torch.nn.init.normal_(temp.weight,mean=0,std=1)
  def forward(self,x):
    return self.features(x)
net=dnn([118*3,118*5,118*5,118*10,118*10,118*10,118*10,118]).to(device)
print('number of params: %d'%(sum(temp.numel() for temp in net.parameters() if temp.requires_grad)))

number of params: 5587418


In [ ]:
# threshold function for p_g
class my_gen_pred_binary(nn.Module):
  def __init__(self):
    super(my_gen_pred_binary,self).__init__()
  def forward(self,x,thresh):
    # thresh=torch.tensor(thresh).double()
    # print(thresh,thresh.dtype)
    # print(x.dtype,x)
    thresh=torch.tensor(thresh).double()
    x=x.double()
    output=torch.where(x<-thresh,-1.,x)
    output=torch.where(output>thresh,1.,output)
    output=torch.where(output*output<thresh*thresh,output/thresh,output)
    return output/2+0.5

## define f-feasibility regularizer 

gen_pred_binary_cell = my_gen_pred_binary() # threshold function
def f_feas_reg(train_batch,pred_label,binary_thres,S,f_max):
  # train_batch,pred_label in device
  # params.to(device)

  # train_batch[:,2,:] # generator limit
  label1 = torch.squeeze(pred_label,1).transpose(0,1) # LMP: 118*batch 
  # train_batch[:,1,:] # generator cost
  profit = label1 - train_batch[:,1::3].transpose(0,1) # profit

  gen_pred_binary = gen_pred_binary_cell(profit,binary_thres)
  p_inj = torch.mul(gen_pred_binary,train_batch[:,2::3].transpose(0,1)) # p_g
  # nodal injection = gen - load
  bus_inj = p_inj - train_batch[:,0::3].transpose(0,1)
  f = torch.matmul(S,bus_inj)
  f=torch.abs(f)
  # do not use torch.max
  f_penalty=torch.where(f>f_max,f-f_max,0.)
  # f_penalty = torch.max(f - f_max,0) # linear penalty
  # f_penalty = torch.exp(f - f_max) - 1 # exponential penalty
  return f_penalty


# define loss function
loss_func=nn.MSELoss()
def my_loss_func(pred,label,W1,train_batch,S,f_max):
  lmda1=torch.from_numpy(np.array([1])).to(device) # L_2
  lmda2=torch.from_numpy(np.array([0.01])).to(device) # L_inf
  # lmda3=torch.from_numpy(np.array([0.02])).to(device) # locality
  lmda4=torch.from_numpy(np.array([0.001])).to(device) # f-feasibility

  binary_thres = 0.97 # P_g threshold for 0
  binary_thres = torch.tensor(binary_thres).to(device)    

  mse = loss_func(pred,label)
  linf = (pred-label).norm(p=float('inf'))
  # locality = torch.matmul(torch.matmul(pred,W1),pred.transpose(1,2))
  f_penalty = f_feas_reg(train_batch,pred,binary_thres,S,f_max)
  # print(pred.shape,pred.transpose(1,2).shape)
  return lmda1*mse + lmda2*linf +lmda4*torch.sum(f_penalty) #+ lmda3*torch.mean(locality) # mean/sum over batch

In [ ]:
!nvidia-smi

Sun Feb 28 23:14:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    31W / 250W |    871MiB / 16280MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_loss=[]
val_loss=[]
optimizer=torch.optim.Adam(net.parameters())
# loss_func=nn.MSELoss()
epochs=500
val_epoch=5

# early stop
tolerance=5
min_delta=1e-4
previous=0
W_tensor = torch.from_numpy(W).float().to(device)
t0=time()
for epoch in range(epochs):
  epoch_loss=0.0
  for x_batch_train,y_batch_train in train_set:
    optimizer.zero_grad()
    output=net(x_batch_train)
    # loss=loss_func(output,y_batch_train)
    loss=my_loss_func(output,y_batch_train,W_tensor,x_batch_train,S,f_max)
    loss.backward()
    epoch_loss+=loss.item()
    # update parameters of net
    optimizer.step()
  train_loss.append(epoch_loss/len(train_set.dataset))
  print("Epoch %d | Train loss: %.4f"%(epoch,train_loss[-1]))
  # val
  if epoch%val_epoch==0:
    net.eval()
    epoch_loss=0.0
    for x_batch_val,y_batch_val in val_set:
      output=net(x_batch_val)
      loss=loss_func(output,y_batch_val)
      epoch_loss+=loss.item()
    val_avg=epoch_loss/len(val_set.dataset)
    if (epoch==0): previous=val_avg
    else:
      if previous-val_avg<min_delta:
        tolerance-=1
        if tolerance==0:
          break
      previous=val_avg
    print("Epoch %d | Eval loss: %.4f" %(epoch,val_avg))
    val_loss.append([epoch,val_avg])
    net.train()
  # validation loop
  with torch.set_grad_enabled(False):
      pass
t1=time()
print('Training time: %.4f'%(t1-t0))
path='/content/drive/MyDrive/gnn/dnn_pytorch/sample_dnn.pickle'
torch.save(net.state_dict(),path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Epoch 0 | Train loss: 2.7624
Epoch 0 | Eval loss: 0.0188
Epoch 1 | Train loss: 2.7574
Epoch 2 | Train loss: 2.7575
Epoch 3 | Train loss: 2.7572
Epoch 4 | Train loss: 2.7571
Epoch 5 | Train loss: 2.7572
Epoch 5 | Eval loss: 6.4748
Epoch 6 | Train loss: 2.7572
Epoch 7 | Train loss: 2.7572
Epoch 8 | Train loss: 2.7571
Epoch 9 | Train loss: 2.7572
Epoch 10 | Train loss: 2.7574
Epoch 10 | Eval loss: 6.0767
Epoch 11 | Train loss: 2.7571
Epoch 12 | Train loss: 2.7571
Epoch 13 | Train loss: 2.7568
Epoch 14 | Train loss: 2.7567
Epoch 15 | Train loss: 2.7569
Epoch 15 | Eval loss: 5.9349
Epoch 16 | Train loss: 2.7568
Epoch 17 | Train loss: 2.7569
Epoch 18 | Train loss: 2.7571
Epoch 19 | Train loss: 2.7567
Epoch 20 | Train loss: 2.7564
Epoch 20 | Eval loss: 6.3929
Epoch 21 | Train loss: 2.7566
Epoch 22 | Train loss: 2.7565
Epoch 23 | Train loss: 2.7564
Epoch 24 | Train loss: 2.7564
Epoch 25 | Train loss: 2.7563
Epoch 25 | Eval loss: 6.4759
Epoch 26 | Train loss: 2.7562
Epoch 27 | Train loss: 2.756

In [ ]:
# plot loss
val_len = len(val_loss)
print(val_len)
val_plt = np.zeros((2,val_len))
for i in range(val_len):
  val_plt[0,i] = val_loss[i][0]
  val_plt[1,i] = val_loss[i][1]

plt.figure()
plot_idx = np.arange(np.size(train_loss))
# plt.plot(plot_idx,loss_optm,lw=2,label='training loss')
# plt.plot(val_plt[0,:],val_plt[1,:],lw=2,label='validation loss')
plt.plot(plot_idx[5:-1],train_loss[5:-1],lw=2,label='training loss')
plt.plot(val_plt[0,1:],val_plt[1,1:],lw=2,label='validation loss')
plt.yscale("log")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show(block=False)

In [ ]:
test_feed=torch.from_numpy(x_test).float().to(device)
output=net(test_feed)
y_pred=output.cpu().detach().numpy()

In [ ]:
n_test=y_test.shape[0]
err_L2=np.zeros(n_test)
err_Linf=np.zeros(n_test)
for i in range(n_test):
  err_L2[i]=np.linalg.norm(y_test[i,:]-y_pred[i,:])/np.linalg.norm(y_test[i,:])
  err_Linf[i]=np.max(np.abs(y_test[i,:]-y_pred[i,:]))/np.max(np.abs(y_test[i,:]))

# filter outliers
L2_thresh=np.argsort(err_L2)[:-50]
Linf_thresh=np.argsort(err_Linf)[:-50]
err_L2_filter=[err_L2[temp] for temp in range(len(err_L2)) if temp in L2_thresh]
err_Linf_filter=[err_Linf[temp] for temp in range(len(err_Linf)) if temp in Linf_thresh]

err_L2_mean = np.mean(err_L2_filter)
err_Linf_mean = np.mean(err_Linf_filter)
print('L2 mean:', err_L2_mean,'L_inf mean:', err_Linf_mean )

fig2 = plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
# plt.hist(np.abs(ga),bins = 10)
plt.plot(err_L2_filter,'bo',markersize=0.5,label = 'L2 error')
plt.plot(err_Linf_filter,'r^',markersize=0.5,label = 'Linf error')
plt.legend(loc="upper right")
plt.xlabel('sample index')
plt.ylabel('error')
plt.title('Normalized sample error')
plt.grid(True)
# error histogram
plt.subplot(1, 2, 2)
plt.hist(err_L2_filter, bins = 50, facecolor='b', alpha=0.75,label = 'L2 error')
plt.hist(err_Linf_filter, bins = 50, facecolor='g', alpha=0.75,label = 'Linf error')
plt.legend(loc="upper right")
plt.xlabel('sample error value')
plt.ylabel('frequency')
plt.title('Histogram of L_2 error')
# plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.grid(True)
plt.show()

# Predict generation using $\pi$
* Using predicted $\pi$ and find the active constraints in $p_G(i)$
* For inactive $p_G(i)$ consider other methods like power flow balance

In [ ]:
gen_limit0 = x[:,0::3].copy()
print(gen_limit0.shape)
n_bus=118
gen_idx = []
for i in range(n_bus):
  if gen_limit0[i,0] > 0:
    gen_idx.append(i)
print(type(gen_idx),len(gen_idx),gen_idx)

In [ ]:
# # get the generator index
# # generator cost data
# gen_cost0
# # generator limit data
# gen_limit0
# # LMP data
# lmp_data
# # generation data
# gen_data

load_data = x.copy()
print(load_data.shape)
n_sample = load_data.shape[0]

x_val_feed = torch.from_numpy(load_data).float()
x_val_feed = x_val_feed.to(device)
print('Dataset size:',x_val_feed.shape)
print('Number of validation points:: ',n_sample)

y_pred = net(x_val_feed) # predict the corredponding LMP

y_pred1 = y_pred.cpu().detach()
print('output size',y_pred1.shape)
y_pred1 = torch.squeeze(y_pred1,1).numpy()
print('reshaped size',y_pred1.shape)

* Save results

In [ ]:
import pickle

In [ ]:
# dataset = {'lmp_pred': y_pred1, 'lmp_true': lmp_data}
# print('Pred:',y_pred1.shape,'True:',lmp_data.shape)

# file_name = 'dc118_lmp_prediction_p10'
# file_path = 'drive/My Drive/gnn/data/results/'
# file_dir = file_path + file_name + '.pickle'
# outfile = open(file_dir, 'wb')
# pickle.dump(dataset, outfile)
# outfile.close()

In [ ]:
gen_cost0 = x[:,1::3].copy()
lmp_data = y.copy()

profit_pred = (y_pred1 - gen_cost0).transpose()
print(np.min(np.abs(profit_pred)))
profit_true = (lmp_data - gen_cost0).transpose()
print(np.min(np.abs(profit_true)))


In [ ]:
p_pred_sort = np.reshape(profit_pred,n_bus*n_sample)
p_true_sort = np.reshape(profit_true,n_bus*n_sample)
print(p_pred_sort.shape)

In [ ]:
print(np.min(p_pred_sort),np.min(p_true_sort))

In [ ]:
fig2 = plt.figure(figsize=(8, 8))
plt.hist(p_pred_sort, bins = 50, facecolor='b', alpha=0.75,label = 'pred. profit')
plt.hist(p_true_sort, bins = 50, facecolor='g', alpha=0.75,label = 'true profit')
plt.legend(loc="upper right")
plt.xlabel('value')
plt.ylabel('frequency')
plt.title('profit histogram')
# plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.grid(True)
plt.show()

In [ ]:
gen_pred_binary = np.zeros((len(gen_idx),n_sample))
gen_true_binary = np.zeros((len(gen_idx),n_sample))
print(gen_pred_binary.shape)

binary_thres = 2.5
binary_thres_true = 1e-5
for i in range(n_sample):
  for j in range(len(gen_idx)):
    # predicted generator limit
    if profit_pred[gen_idx[j],i] > binary_thres:
      gen_pred_binary[j,i] = 1
    elif profit_pred[gen_idx[j],i] < -binary_thres:
      gen_pred_binary[j,i] = 0
    else:
      gen_pred_binary[j,i] = (profit_pred[gen_idx[j],i] + binary_thres) / (2*binary_thres)
    # true generator limit
    if profit_true[gen_idx[j],i] > binary_thres_true:
      gen_true_binary[j,i] = 1
    elif profit_true[gen_idx[j],i] < -binary_thres_true:
      gen_true_binary[j,i] = 0
    else:
      gen_true_binary[j,i] = 0.5

gen_binary_err = np.abs(gen_true_binary - gen_pred_binary)
print('max binary error:',np.max(gen_binary_err))
# count the wrong entries
gen_binary_err_ct = np.sum(gen_binary_err)
gen_binary_err_ratio = gen_binary_err_ct / (len(gen_idx)*n_sample)
print('Binary accuracy:',1-gen_binary_err_ratio)

In [ ]:
print(gen_limit0.shape)
print(np.transpose(x_test).shape,np.transpose(load_data).shape)

 # Test flow feasibility

In [ ]:
# x = [load, gen_cost, gen_lim]

binary_thres = 0.97
binary_thres_true = 1e-5

gen_pred_binary_full = np.zeros((n_bus,n_sample))
gen_true_binary_full = np.zeros((n_bus,n_sample))
# print(gen_pred_binary.shape,profit_pred.shape,gen_pred_binary_full.shape)

for i in range(n_sample):
  for j in range(len(gen_idx)):
    # predicted generator limit
    if profit_pred[gen_idx[j],i] > binary_thres:
      gen_pred_binary_full[gen_idx[j],i] = 1
    elif profit_pred[gen_idx[j],i] < -binary_thres:
      gen_pred_binary_full[gen_idx[j],i] = 0
    else:
      gen_pred_binary_full[gen_idx[j],i] = 0.5 # this is only an approximation
    # true generator limit
    if profit_true[gen_idx[j],i] > binary_thres_true:
      gen_true_binary_full[gen_idx[j],i] = 1
    elif profit_true[gen_idx[j],i] < -binary_thres_true:
      gen_true_binary_full[gen_idx[j],i] = 0
    else:
      gen_true_binary_full[gen_idx[j],i] = 0.5

# injection by generators
gen_inj = np.multiply(gen_pred_binary_full,gen_limit0.transpose())
gen_inj_true = np.multiply(gen_true_binary_full,gen_limit0.transpose())
# nodal injection
load0 = x[:,0::3].copy().transpose() # load file
p_inj = gen_inj - load0
p_inj_true = gen_inj_true - load0

In [ ]:
# load0 = x[:,0,:].copy()
print(np.sum(p_inj_true),np.sum(gen_inj_true))
print(np.sum(p_inj),np.sum(load0),np.sum(gen_inj))

* Calculate flow

In [ ]:
n_line = np.size(S_isf,0)
flow_est = np.zeros((n_line,n_sample))
flow_est0 = np.zeros((n_line,n_sample))

f_binary = np.zeros((n_line,n_sample))
f_binary0 = np.zeros((n_line,n_sample))

# for i in range(n_sample):
flow_est = np.dot(S_isf,p_inj)
flow_est0 = np.dot(S_isf,p_inj_true)
# f_max
f_max = f_max.cpu().detach().numpy()
f_binary = (np.abs(flow_est)-f_max > 0)
f_binary0 = (np.abs(flow_est0)-f_max > 0)

In [ ]:
f_tot_sample = n_line * n_sample
print(np.sum(f_binary),np.sum(f_binary0))
print(np.sum(f_binary)/f_tot_sample,np.sum(f_binary0)/f_tot_sample)

In [ ]:
a= np.asarray([[1,2,1],[2,2,1]])
b = np.asarray([1,1,1])
c = (a-b>0)
print(c)
print(np.sum(c))

In [ ]:
f_line = np.sum(f_binary,0)
f_samp = np.sum(f_binary,1)
print('max sample pred:',np.max(f_line))
print('max line pred:',np.max(f_samp))

f_line0  = np.sum(f_binary0,0)
f_samp0 = np.sum(f_binary0,1)
print('max sample true:',np.max(f_line0))
print('max line true:',np.max(f_samp0))

